<a href="https://colab.research.google.com/github/BramarambikaNandyala/PROJECTS/blob/master/Expense_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from datetime import datetime

class ExpenseTracker:
    def __init__(self, db_name):
        self.db_name = db_name
        self.conn = sqlite3.connect(db_name)
        self.create_table()

    def create_table(self):
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS expenses (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    category TEXT NOT NULL,
                    amount REAL NOT NULL,
                    date DATE NOT NULL
                )
            ''')
            self.conn.commit()
        except sqlite3.Error as e:
            print(f"Error creating table: {e}")

    def add_expense(self, category, amount):
        try:
            cursor = self.conn.cursor()
            date = datetime.now().date()
            cursor.execute('''
                INSERT INTO expenses (category, amount, date)
                VALUES (?, ?, ?)
            ''', (category, amount, date))
            self.conn.commit()
            print("Expense added successfully!")
        except sqlite3.Error as e:
            print(f"Error adding expense: {e}")

    def get_categories(self):
        categories = ["food", "entertainment", "transportation", "medication"]
        return categories

    def get_monthly_expenses(self):
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                SELECT strftime('%Y-%m', date) AS month, category, COALESCE(SUM(amount), 0) AS total_amount
                FROM expenses
                GROUP BY month, category
            ''')
            rows = cursor.fetchall()
            if not rows:
                print("No expenses recorded yet.")
                return

            # Collect unique categories and months
            categories = self.get_categories()
            months = sorted(set(row[0] for row in rows))

            # Print expenses for each month
            for month in months:
                print(f"\nMonthly Expenses for {month}:")
                for category in categories:
                    total_amount = sum(row[2] for row in rows if row[0] == month and row[1] == category)
                    print(f"{category}: {total_amount}")
        except sqlite3.Error as e:
            print(f"Error fetching monthly expenses: {e}")

    def close_connection(self):
        try:
            self.conn.close()
            print("Database connection closed.")
        except sqlite3.Error as e:
            print(f"Error closing connection: {e}")

def main():
    tracker = ExpenseTracker("expenses.db")

    while True:
        print("\n1. Add Expense")
        print("2. Monthly Expenses")
        print("3. Exit")

        try:
            choice = int(input("Enter your choice: "))

            if choice == 1:
                categories = tracker.get_categories()
                print("Enter expense for the following categories:")
                for category in categories:
                    amount_entered = False
                    while not amount_entered:
                        try:
                            amount = float(input(f"{category}: ") or 0)
                            tracker.add_expense(category, amount)
                            amount_entered = True  # Set flag to True after entering amount
                        except ValueError:
                            print("Invalid input. Please enter a valid amount.")
            elif choice == 2:
                tracker.get_monthly_expenses()
            elif choice == 3:
                print("Exiting...")
                tracker.close_connection()
                break
            else:
                print("Invalid choice. Please choose again.")
        except ValueError:
            print("Invalid input. Please enter a number for the choice.")

if __name__ == "__main__":
    main()



1. Add Expense
2. Monthly Expenses
3. Exit
